## PyTorch ile Basit MultiHeadSelfAttention (Başlangıç);

* Aşağıdaki hücre, teoride anlattığımız adımları takip eden, açıklamalı ve test edilebilir bir PyTorch implementasyonudur. Bu sürüm eğitim ve geliştirme için okunaklı ve genişletilebilir tutuldu — sonraki adımlarda performans optimizasyonu, flash attention, qkv tek matris haline getirme, rotary embedding, ya da causal masking için ilerleyeceğiz.

In [2]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F

In [3]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model : int ,num_heads:int=8 , dp:float = 0.3 ):
        super().__init__()
        assert d_model % num_heads == 0 , "d_model num_heads'e tam bölünmelidir"
        self.d_model = d_model
        self.num_heads = num_heads
        self.d_k = d_model // num_heads

        self.q_lin = nn.Linear(d_model , d_model)
        self.v_lin = nn.Linear(d_model , d_model)
        self.k_lin = nn.Linear(d_model , d_model)

        self.out_lin = nn.Linear(d_model , d_model)

        self.dropout = nn.Dropout(dp)

    def forward(self,x:torch.Tensor , mask:torch.Tensor = None):
        b,n,d = x.size()
        q = self.q_lin(x).view(b,n,self.num_heads , self.d_k).transpose(1,2)
        k = self.k_lin(x).view(b,n,self.num_heads , self.d_k).transpose(1,2)
        v = self.v_lin(x).view(b,n,self.num_heads , self.d_k).transpose(1,2)

        scores = torch.matmul(q,k.transpose(-2,-1)) / math.sqrt(self.d_k)

        if mask is not None:
            if mask.dim() == 2:
                mask = mask.unsqueeze(1).unsqueeze(1)
            elif mask.dim() == 3:
                mask = mask.unsqueeze(1)
            scores = scores.masked_fill_(mask == 0 , float("-inf"))
        
        attn= F.softmax(scores , dim= -1)
        attn = self.dropout(attn)

        out = torch.matmul(attn,v)
        out = out.transpose(1,2).contiguous().view(b,n,d)
        out = self.out_lin(out)
        return out , attn

In [5]:
if __name__ == '__main__':
    torch.manual_seed(0)
    batch, seq_len, d_model = 2, 5, 64
    x = torch.randn(batch, seq_len, d_model)
    model = MultiHeadAttention(d_model=d_model, num_heads=8, dp=0.)
    out, attn = model(x)
    print('out.shape =', out.shape) # beklenen: (2,5,64)
    print('attn.shape =', attn.shape) # beklenen: (2,8,5,5)


# padding mask örneği
    mask = torch.tensor([[1,1,1,0,0],[1,1,1,1,0]])
    out_masked, attn_masked = model(x, mask=mask)
    print('masked attn sum row (first batch, first head):', attn_masked[0,0,0].sum().item())

out.shape = torch.Size([2, 5, 64])
attn.shape = torch.Size([2, 8, 5, 5])
masked attn sum row (first batch, first head): 1.0000001192092896


### Şu an elimizde:




✅ Ayrı Q, K, V lineer katmanları var.

✅ Dropout ve padding mask desteği eklendi.

✅ Dikkat skorları ölçekleniyor ve softmax sonrası normalize ediliyor.

✅ Multi-Head yapısı dinamik ve testle doğrulandı.

## 🔍 Multi-Head Attention — Kod Özeti

1. **Linear Projeksiyonlar:**  
   Girdi (x) → `q_proj`, `k_proj`, `v_proj` katmanlarıyla sırasıyla **Query (Q)**, **Key (K)**, **Value (V)** matrislerine dönüştürülür.

2. **Head Bölünmesi:**  
   `embed_dim`, `num_heads`’e bölünür → her head’in boyutu `head_dim = embed_dim // num_heads` olur.  
   Tensorlar `(batch, seq_len, num_heads, head_dim)` biçimine getirilir.

3. **Skor Hesabı:**  
   Her head için dikkat skorları hesaplanır:  
   \[
   \text{scores} = \frac{QK^T}{\sqrt{head\_dim}}
   \]
   Daha sonra `Softmax` uygulanarak olasılık dağılımı elde edilir.

4. **Ağırlıklı Toplama:**  
   Bu dağılım `V` ile çarpılarak her head’in çıktısı bulunur:
   \[
   \text{head\_output} = \text{Softmax(scores)} \times V
   \]

5. **Birleştirme ve Lineer Çıkış:**  
   Tüm head çıktıları `concat` edilir ve `out_proj` katmanıyla tekrar `embed_dim` boyutuna dönüştürülür.

6. **Opsiyonel Dropout / Norm:**  
   Eğitim sırasında aşırı öğrenmeyi önlemek için Dropout ve LayerNorm eklenebilir.


---
## Şimdi bu attention mekanizmasını daha da ileri taşıyalım. ;
* Q/K/V projeksiyonlarını tek lineer katmanda birleştirme (W_qkv → performans artışı).

* Causal mask (look-ahead) ekleyerek dil modelleme ve autoregressive görevler için uygun hale getirme
---

## ⚙️ Geliştirme 1 — Q/K/V'yi Tek Lineer Katmanda Birleştirme

Normalde `q_proj`, `k_proj`, `v_proj` olmak üzere 3 ayrı `nn.Linear` kullanılır.  
Ancak performans kazanmak için **tek bir büyük ağırlık matrisi (W_qkv)** ile bu işlemler birleştirilebilir.

\[
[Q, K, V] = xW_{qkv}
\]

Bu yaklaşım:
- Hesaplama maliyetini azaltır (tek matris çarpımı),
- GPU optimizasyonundan daha iyi faydalanır.


## ⚙️ Geliştirme 2 — Causal Mask (Look-Ahead)

Dil modelleme ve autoregressive görevlerde, modelin **gelecekteki token’ları görmemesi** gerekir.  
Bu amaçla **causal mask** (üçgensel maske) uygulanır:

\[
mask_{ij} = 
\begin{cases}
0, & \text{if } j \le i \\
-\infty, & \text{if } j > i
\end{cases}
\]

Softmax öncesinde bu maske eklenir:
\[
\text{scores} = \frac{QK^T}{\sqrt{d_k}} + \text{mask}
\]


In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class MultiHeadAttention(nn.Module):
    def __init__(self, embed_dim: int, num_heads: int = 8, dp: float = 0.1, causal: bool = False):
        super().__init__()
        assert embed_dim % num_heads == 0, "embed_dim must be divisible by num_heads"
        
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.head_dim = embed_dim // num_heads
        self.causal = causal
        
        # 🔹 Q/K/V birleşik projeksiyon katmanı
        self.qkv_proj = nn.Linear(embed_dim, embed_dim * 3)
        self.out_proj = nn.Linear(embed_dim, embed_dim)
        self.dropout = nn.Dropout(dp)

    def forward(self, x):
        B, L, E = x.shape
        
        # 1️⃣ Tek lineer katmandan Q, K, V üretimi
        qkv = self.qkv_proj(x)  # (B, L, 3E)
        Q, K, V = qkv.chunk(3, dim=-1)  # Üç parçaya ayır
        
        # 2️⃣ Head bölünmesi
        def reshape_heads(t):
            return t.view(B, L, self.num_heads, self.head_dim).transpose(1, 2)
        Q, K, V = map(reshape_heads, (Q, K, V))
        
        # 3️⃣ Skor hesabı
        scores = torch.matmul(Q, K.transpose(-2, -1)) / (self.head_dim ** 0.5)
        
        # 4️⃣ Causal Mask ekleme (look-ahead)
        if self.causal:
            mask = torch.triu(torch.ones(L, L, device=x.device), diagonal=1).bool()
            scores = scores.masked_fill(mask, float('-inf'))
        
        attn = F.softmax(scores, dim=-1)
        attn = self.dropout(attn)
        
        out = torch.matmul(attn, V)  # (B, num_heads, L, head_dim)
        out = out.transpose(1, 2).contiguous().view(B, L, E)
        return self.out_proj(out)

# Test
x = torch.randn(2, 10, 512)
mha = MultiHeadAttention(embed_dim=512, num_heads=8, causal=True)
y = mha(x)
print(y.shape)


torch.Size([2, 10, 512])


---

## ⚙️ Aşama 3 — Residual Bağlantı + Layer Normalization

### 🔹 Residual Connection
Multi-Head Attention bloğu, giriş bilgisini tamamen kaybetmesin diye:
\[
y = x + \text{Dropout}(\text{Attention}(x))
\]
şeklinde geri eklenir.  
Bu, **gradyan akışını kolaylaştırır** ve **öğrenmeyi stabilize eder**.

### 🔹 Layer Normalization
Residual toplama sonrası, modelin çıktısını normalize ederek daha kararlı hale getirir:
\[
\text{output} = \text{LayerNorm}(y)
\]

Bu iki adım sayesinde dikkat bloğu, **Transformer Encoder/Decoder** yapılarında doğrudan kullanılabilir hale gelir.


---

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class MultiHeadAttention(nn.Module):
    def __init__(self, embed_dim: int, num_heads: int = 8, dp: float = 0.1, causal: bool = False):
        super().__init__()
        assert embed_dim % num_heads == 0, "embed_dim must be divisible by num_heads"
        
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.head_dim = embed_dim // num_heads
        self.causal = causal
        
        # Tek lineer katman (Q, K, V birleşik)
        self.qkv_proj = nn.Linear(embed_dim, embed_dim * 3)
        self.out_proj = nn.Linear(embed_dim, embed_dim)
        
        # Dropout ve Normalization
        self.dropout = nn.Dropout(dp)
        self.layernorm = nn.LayerNorm(embed_dim)

    def forward(self, x):
        B, L, E = x.shape
        
        # --- Q, K, V projeksiyonu ---
        qkv = self.qkv_proj(x)
        Q, K, V = qkv.chunk(3, dim=-1)
        
        # --- Head bölünmesi ---
        def reshape_heads(t):
            return t.view(B, L, self.num_heads, self.head_dim).transpose(1, 2)
        Q, K, V = map(reshape_heads, (Q, K, V))
        
        # --- Skor hesabı ---
        scores = torch.matmul(Q, K.transpose(-2, -1)) / (self.head_dim ** 0.5)
        
        # --- Causal mask ---
        if self.causal:
            mask = torch.triu(torch.ones(L, L, device=x.device), diagonal=1).bool()
            scores = scores.masked_fill(mask, float('-inf'))
        
        # --- Attention ağırlıkları ---
        attn = F.softmax(scores, dim=-1)
        attn = self.dropout(attn)
        
        # --- Ağırlıklı toplama ---
        out = torch.matmul(attn, V)
        out = out.transpose(1, 2).contiguous().view(B, L, E)
        out = self.out_proj(out)
        
        # --- Residual + Normalization ---
        x = x + self.dropout(out)
        x = self.layernorm(x)
        return x

# Test
x = torch.randn(2, 10, 512)
mha = MultiHeadAttention(embed_dim=512, num_heads=8, causal=True)
y = mha(x)
print(y.shape)


torch.Size([2, 10, 512])
